In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras import backend as K
from matplotlib import pyplot
import os
!pip install pypianoroll
import pypianoroll


     |████████████████████████████████| 5.6MB 9.3MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp37-none-any.whl size=5591954 sha256=833ab431bcbe2354d9fa339d59ae81a7e7c18507adf3f131c3f889beac58a9dd
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
Successfully built pretty-midi


In [3]:
dataset_name = 'dataset_np_2.0.npy'
path = '/content/gdrive/MyDrive/Colab Notebooks/senior_research/version 2.0/'
path_progression = path+'GAN_progression/'
path_model_backups = path+'GAN_models/'
path_dataset = '/content/gdrive/MyDrive/Colab Notebooks/senior_research/'+dataset_name

In [4]:

# load and prepare mnist training images
def load_real_samples():
    # load midi dataset
    X = np.load(path_dataset)
    return X

# load image data
dataset = load_real_samples()

In [5]:
def sigmoid128(x, beta=1.0):
    return 128 * K.sigmoid(beta * x)

def define_discriminator(in_shape=(2,1024,128)):
    model = Sequential()
    model.add(Conv2D(64, (5,5), strides=(1, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, (5,5), strides=(2, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, (5,5), strides=(2, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(64, (3,3), strides=(1, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def define_generator(latent_dim):
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 32 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((128, 32, 4)))

    model.add(Conv2DTranspose(64, (5,5), strides=(1,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(64, (5,5), strides=(2,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(64, (5,5), strides=(2,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (5,5), strides=(1,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (5,5), strides=(2,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (5,5), strides=(1,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (5,5), strides=(2,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (5,5), strides=(1,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (5,5), strides=(2,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (5,5), strides=(1,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (5,5), strides=(2,1), activation='sigmoid', padding='same'))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected midis
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples) 
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)#[:,:,:,:,0]
    #X = X*128
    X = np.floor(np.where(X < 30, 0, X))
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

# create and save a plot of generated images (reversed grayscale)
def save_midis(examples, epoch, n=1):
    # save midis
    # midis = examples*128
    # midis = np.floor(np.where(midis < 1, 0, midis))
    midis=examples
    for i in range(n):
        name = 'generated_midi_e%03d_n%03d' % (epoch+1, i+1)
        #initialize
        multitrack = pypianoroll.Multitrack(name=name)
        #create left and right tracks
        right_track = pypianoroll.StandardTrack(name='right',pianoroll=midis[i,0])
        left_track = pypianoroll.StandardTrack(name='left',pianoroll=midis[i,1])
        #add tracks
        multitrack.append(right_track)
        multitrack.append(left_track)
        #save
        pypianoroll.write(path_progression+name+'.mid',multitrack)

# size of the latent space
latent_dim = 300
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)



In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=4):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    # for i in range(1):
    #     # enumerate batches over the training set
    #     for j in range(5):
    #         # get randomly selected 'real' samples
    #         X_real, y_real = generate_real_samples(dataset, half_batch)
    #         # generate 'fake' examples
    #         X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
    #         # create training set for the discriminator
    #         X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
    #         # prepare points in latent space as input for the generator
    #         X_gan = generate_latent_points(latent_dim, n_batch)
    #         # create inverted labels for the fake samples
    #         y_gan = ones((n_batch, 1))
    #         # update the generator via the discriminator's error
    #         g_loss = gan_model.train_on_batch(X_gan, y_gan)

    #         print('>%d, %d/%d, d=%.3f, g=%.3f' % (-1, j+1, bat_per_epo, -1, g_loss))

    # manually enumerate epochs
    for i in range(bat_per_epo):
        # enumerate batches over the training set
        g_d_loss_ratio = 1
        count = 0
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # create training set for the discriminator
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # update discriminator model weights

            if count % int(g_d_loss_ratio)*3 == 0:
            #if True:
                d_loss, _ = d_model.train_on_batch(X, y)
                g_d_loss_ratio = g_loss / d_loss
                if g_d_loss_ratio < 1:
                    g_d_loss_ratio = 1
                count=0
                #d_loss, _ = d_model.train_on_batch(X[:,:,:,:,np.newaxis], y)
                # summarize loss on this batch
                print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
                # evaluate the model performance, sometimes
                if j == 0:
                    losses[i,0] = d_loss
                    losses[i,1] = g_loss
            count+=1
        if (i+1) % 1 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=10, n_samples_save=1):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save midis
    save_midis(x_fake, epoch, n_samples_save)
    # save the generator and discriminator models
    g_filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(os.path.join(path_model_backups,g_filename))
    d_filename = 'discriminator_model_%03d.h5' % (epoch + 1)
    d_model.save(os.path.join(path_model_backups,d_filename))

n_epochs=500
losses = np.zeros((n_epochs,2))
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=n_epochs, n_batch=10)

>-1, 1/288, d=-1.000, g=0.694
>-1, 2/288, d=-1.000, g=0.694
>-1, 3/288, d=-1.000, g=0.694
>-1, 4/288, d=-1.000, g=0.694
>-1, 5/288, d=-1.000, g=0.694
>1, 1/288, d=0.668, g=0.693
>1, 2/288, d=0.353, g=0.500
>1, 3/288, d=0.348, g=0.414
>1, 4/288, d=0.346, g=0.366
>1, 5/288, d=0.345, g=0.288
>1, 6/288, d=0.345, g=0.030
>1, 7/288, d=0.344, g=0.015
>1, 8/288, d=0.343, g=0.012
>1, 9/288, d=0.342, g=0.011
>1, 10/288, d=0.341, g=0.010
>1, 11/288, d=0.339, g=0.010
>1, 12/288, d=0.338, g=0.010
>1, 13/288, d=0.336, g=0.010
>1, 14/288, d=0.332, g=0.009
>1, 15/288, d=0.330, g=0.009
>1, 16/288, d=0.324, g=0.008
>1, 17/288, d=0.318, g=0.008
>1, 18/288, d=0.311, g=0.007
>1, 19/288, d=0.303, g=0.007
>1, 20/288, d=0.292, g=0.007
>1, 21/288, d=0.279, g=0.007
>1, 22/288, d=0.264, g=0.007
>1, 23/288, d=0.246, g=0.007
>1, 24/288, d=0.226, g=0.007
>1, 25/288, d=0.204, g=0.007
>1, 26/288, d=0.179, g=0.007
>1, 27/288, d=0.154, g=0.008
>1, 28/288, d=0.128, g=0.009
>1, 29/288, d=0.104, g=0.011
>1, 30/288, d=0.08

KeyboardInterrupt: ignored

In [ ]:
def save_midis(examples, epoch, n=1):
    # save midis
    midis = examples*128
    midis = np.floor(np.where(midis < 1, 0, midis))
    for i in range(n):
        name = 'generated_midi_e%03d_n%03d' % (epoch+1, i+1)
        #initialize
        multitrack = pypianoroll.Multitrack(name=name)
        #create left and right tracks
        right_track = pypianoroll.StandardTrack(name='right',pianoroll=midis[i,0])
        left_track = pypianoroll.StandardTrack(name='left',pianoroll=midis[i,1])
        #add tracks
        multitrack.append(right_track)
        multitrack.append(left_track)
        #save
        pypianoroll.write(path_progression+name+'.mid',multitrack)
X_fake, y_fake = generate_fake_samples(g_model, 100, 1)
print(X_fake.shape)
save_midis(X_fake,0,1)
np.save('test',X_fake)

(1, 2, 1024, 128)
